In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import glob
import matplotlib.pyplot as plt

In [29]:
def unify_csv(path, name):
    all_files = glob.glob(path + "*.csv")
    if name not in["exodos", "junker2"]: # the exodos folder seem to contain weird symbols for some column names, thus latin-1 encoding is needed
        dfs = [pd.read_csv(file, sep=';') for file in all_files]
    else:
        dfs = [pd.read_csv(file, sep=';', encoding='latin-1') for file in all_files]
    combined_df = pd.concat(dfs, join = 'outer', ignore_index=True)
    # Combine 'Datum' and 'Absolutezeit' into a single datetime column
    combined_df['Timestamp'] = pd.to_datetime(combined_df['Datum'] + ' ' + combined_df['Absolutzeit'],  dayfirst=True)
    # Sort the concatenated DataFrame by the new 'Timestamp' column
    sorted_df = combined_df.sort_values(by='Timestamp')
    # Save the result to a new CSV if needed
    sorted_df.to_csv(name + ".csv", sep = ';', index=True)

# Unify Auto Press folder (inclusive join. High disparity in column lengths among the csv's)

In [15]:
folder_path = "./New_HalcoR_Data(2024)/OneDrive_1_12-5-2024/Auto Press/"
unify_csv(folder_path, "autopress")

In [16]:
# Date range
df = pd.read_csv("autopress.csv", sep = ';')
print("Dates ranging from " + df['Timestamp'][0] + " to " + df['Timestamp'][-1:].iloc[0])

/tmp/ipykernel_574747/3949148293.py:2: DtypeWarning: Columns (6,76,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("autopress.csv", sep = ';')


Dates ranging from 2023-12-07 08:41:41 to 2024-11-11 10:53:00


# Unify Antlies Pressas folder (inclusive join. High disparity in column lengths among the csv's)

In [17]:
folder_path = "./New_HalcoR_Data(2024)/OneDrive_1_12-5-2024/Antlies Pressas/"
unify_csv(folder_path, "antliespressas")

/tmp/ipykernel_574747/1463582008.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, join = 'outer', ignore_index=True)


In [18]:
# Date range
df = pd.read_csv("antliespressas.csv", sep = ';')
print("Dates ranging from " + df['Timestamp'][0] + " to " + df['Timestamp'][-1:].iloc[0])

Dates ranging from 2024-01-02 11:42:59 to 2024-04-17 17:59:37


# Unify Exodos folder (inclusive join. High disparity in column lengths among the csv's)

In [19]:
folder_path = "./New_HalcoR_Data(2024)/OneDrive_1_12-5-2024/Exodos/"
unify_csv(folder_path, "exodos")

/tmp/ipykernel_574747/1463582008.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, join = 'outer', ignore_index=True)


In [20]:
# Date range
df = pd.read_csv("exodos.csv", sep = ';')
print("Dates ranging from " + df['Timestamp'][0] + " to " + df['Timestamp'][-1:].iloc[0])

Dates ranging from 2024-06-06 08:31:51 to 2024-10-25 16:02:42


# Unify InterlockAntliostasiou folder (inclusive join. High disparity in column lengths among the csv's)

In [21]:
folder_path = "./New_HalcoR_Data(2024)/OneDrive_1_12-5-2024/InterlockAntliostaiou/"
unify_csv(folder_path, "interlockantliostaiou")

/tmp/ipykernel_574747/1463582008.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, join = 'outer', ignore_index=True)


In [22]:
# Date range
df = pd.read_csv("interlockantliostaiou.csv", sep = ';')
print("Dates ranging from " + df['Timestamp'][0] + " to " + df['Timestamp'][-1:].iloc[0])

Dates ranging from 2023-12-06 15:43:26 to 2024-06-03 07:20:11


# Unify Junker1 folder (inclusive join. High disparity in column lengths among the csv's)

In [23]:
folder_path = "./New_HalcoR_Data(2024)/OneDrive_1_12-5-2024/Junker1/"
unify_csv(folder_path, "junker1")

In [24]:
# Date range
df = pd.read_csv("junker1.csv", sep = ';')
print("Dates ranging from " + df['Timestamp'][0] + " to " + df['Timestamp'][-1:].iloc[0])

/tmp/ipykernel_574747/1628634808.py:2: DtypeWarning: Columns (103,104,105) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("junker1.csv", sep = ';')


Dates ranging from 2024-06-24 16:24:22 to 2024-10-24 11:09:14


# Unify Junker2 folder (inclusive join. High disparity in column lengths among the csv's)

In [30]:
folder_path = "./New_HalcoR_Data(2024)/OneDrive_1_12-5-2024/Junker2/"
unify_csv(folder_path, "junker2")

In [31]:
# Date range
df = pd.read_csv("junker2.csv", sep = ';')
print("Dates ranging from " + df['Timestamp'][0] + " to " + df['Timestamp'][-1:].iloc[0])

/tmp/ipykernel_574747/3174954517.py:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("junker2.csv", sep = ';')


Dates ranging from 2024-06-28 07:55:20 to 2024-10-25 12:12:12


# Testing the Energy data

In [23]:
# Concatenate all energies together for each timestamp
total_energy = None
# Folder with energy consumption files
path_to_energy_consumption_files = './PRESS PME DATA FY2024.xlsx'
excel_data = pd.ExcelFile(path_to_energy_consumption_files)
start_timestamp = pd.Timestamp('2024-01-01 11:30:00')
df_energy = []
# Loop through each sheet
for sheet_name in excel_data.sheet_names:
    # Read the sheet
    df = excel_data.parse(sheet_name)
    # Keep only the first two columns
    df = df.iloc[:, :2]
    # Make all the sheets start from a specific timestamp for smoothness. Convert the timestamp column to datetime (assume it's the first column)
    df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
    df = df[df.iloc[:, 0] >= start_timestamp]
    df['Real Energy'] = pd.to_numeric(df['Real Energy'], downcast='float', errors='coerce')
    # Adjust timestamps where seconds are not 00
    df['Timestamp'] = df['Timestamp'].apply(lambda x: x.replace(second=0))
    df = df.sort_values(by='Timestamp', ascending= True)
    # Remove duplicates that exist in the dataset, if they exist.
    df.drop_duplicates(subset='Timestamp', keep='last', inplace = True)
    # the '-' rows should be filled with the previous value instead (the previous 15-minute)
    df = df.applymap(lambda x : np.nan if x == '-' else x) # convert - into nan so later can be filled with 'fillna' later
    df.set_index('Timestamp', inplace=True)
    df = df.resample('15min', closed='right', label='right').ffill()
    df.reset_index(inplace=True)
    df['Real Energy'] = df['Real Energy'].interpolate() # interpolate values
    # Append the DataFrame to the list
    df_energy.append(df)


total_energy = pd.concat(df_energy, ignore_index=True)
total_energy = total_energy.groupby('Timestamp').sum().reset_index()
total_energy.set_index('Timestamp', inplace=True)
# Set the initial granularity to 15 minute intervals for baseline.
total_energy = total_energy.resample('15min', closed='right', label='right').ffill()
# total_energy.replace(0, np.nan, inplace=True)
# total_energy = total_energy.interpolate()
total_energy.reset_index(inplace=True)

# Since energy was cumulative, We take the difference and save it into a column
total_energy['diff'] = total_energy['Real Energy'].diff()
total_energy['diff'] = total_energy['diff'].interpolate()
total_energy['Timestamp'] = pd.to_datetime(total_energy['Timestamp'])
total_energy = total_energy.iloc[:-1]
total_energy.drop(columns='Real Energy', inplace = True)

/tmp/ipykernel_313725/4119017451.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x : np.nan if x == '-' else x) # convert - into nan so later can be filled with 'fillna' later
/tmp/ipykernel_313725/4119017451.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x : np.nan if x == '-' else x) # convert - into nan so later can be filled with 'fillna' later
/tmp/ipykernel_313725/4119017451.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x : np.nan if x == '-' else x) # convert - into nan so later can be filled with 'fillna' later
/tmp/ipykernel_313725/4119017451.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x : np.nan if x == '-' else x) # convert - into nan so later can be filled with 'fillna' later
/tmp/ipykernel_313725/411901

In [ ]:
# Plot energy consumption over time
plt.figure(figsize=(10, 6))
plt.plot(total_energy['Timestamp'], total_energy['diff'], marker='o', linestyle='-', color='b', label='Energy Consumption')

# Set title and labels
plt.title('Energy Consumption Over Time', fontsize=16)
plt.xlabel('Timestamp', fontsize=14)
plt.ylabel('Energy Consumption', fontsize=14)

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Add a legend
plt.legend()

# Display the grid
plt.grid(True, linestyle='--', alpha=0.6)

# Show the plot
plt.tight_layout()
plt.show()

In [25]:
total_energy['Timestamp'] = total_energy['Timestamp'].dt.strftime('%d/%m/%Y %H:%M:%S')
total_energy.to_excel("temp.xlsx", engine='openpyxl', index=False)